# GKS 2022 Data Visualization

## Importing libraries


In [14]:
#installation
##### This really depends for every computer, so please be caerful with the instalation
##!pip install konlpy
##!pip install pycountry-convert
##!pip install dataprep
###!pip install geopy

In [50]:
import pandas as pd
from konlpy.tag import Kkma, Hannanum, Komoran
from konlpy.utils import pprint
import pycountry_convert as pc
##import dataprep
from geopy.geocoders import Nominatim #function to get longitude and latitude data from country name

## Importing Data

In [3]:
df_gks = pd.read_csv("2022_GKS_Successful_Candidates.csv")

In [4]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master's,SEOKYEONG UNIVERSITY,Kyungpook National University,Electronic and Electrical Engineering,NaN
1,CS01220219,Embassy,Algeria,HADJ MOHAMMED Hanane,Doctoral,PAI CHAI UNIVERSITY,Seoul National University,Earth and Environmental Sciences,NaN
2,CS01220220,Embassy,Algeria,BOURAHLA Kamelia,Master's,SUNMOON UNIVERSITY,KOREA UNIVERSITY,Business Administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master's,SUNMOON UNIVERSITY,PUSAN NATIONAL UNIVERSITY,Dept. of Food and Resource Economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master's,Keimyung University,Hankuk University of Foreign Studies,International Studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master's,Konyang University,Sogang University,International Relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master's,PAI CHAI UNIVERSITY,Hankuk University of Foreign Studies,Korean Language and Literature,Conditional\r\nSuccessful Candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master's,Busan University of Foreign\r\nStudies,Ewha Womans University,International Studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master's,exempt,Dongguk University,Film and Digital Media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master's,exempt,KOREA UNIVERSITY,Media & Communication,Scholarship Deffered\r\n(6 months)


## Cleaning and processing Data

In [5]:
df_gks.describe()

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
count,1193,1193,1193,1193,1193,1193,1193,1193,219
unique,1193,4,130,1193,5,32,59,459,3
top,CS01220218,Embassy,Indonesia,Agalliu Enrik,Master's,exempt,Hankuk University of Foreign Studies,International Studies,Conditional\r\nSuccessful Candidate
freq,1,506,99,1,929,236,56,44,149


In [6]:
df_gks['Korean Language Institution'] = df_gks['Korean Language Institution'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Korean Language Institution'] = df_gks['Korean Language Institution'].str.title()

df_gks['University'] = df_gks['University'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['University'] = df_gks['University'].str.title()

df_gks['Full Name (Family Given Middle)'] = df_gks['Full Name (Family Given Middle)'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Full Name (Family Given Middle)'] = df_gks['Full Name (Family Given Middle)'].str.title()

df_gks['Scholarship Status'] = df_gks['Scholarship Status'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Scholarship Status'] = df_gks['Scholarship Status'].str.capitalize()

df_gks['Department'] = df_gks['Department'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Department'] = df_gks['Department'].str.capitalize()

df_gks['Country'] = df_gks['Country'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True)
df_gks['Country'] = df_gks['Country'].str.title()

df_gks = df_gks.replace(to_replace="Master's", value="Master")


In [7]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months)


In [8]:
df_gks = df_gks.replace(to_replace="Bosnia Andherzegovin", value="Bosnia and Herzegovina")
df_gks = df_gks.replace(to_replace="Burkinafaso", value="Burkina Faso")
df_gks = df_gks.replace(to_replace="Coted'Ivoire", value="Côte d'Ivoire")
df_gks = df_gks.replace(to_replace="Czechrepublic", value="Czechia")
df_gks = df_gks.replace(to_replace="Dominicanrepublic", value="Dominican Republic")
df_gks = df_gks.replace(to_replace="Dr Congo", value="Congo")
df_gks = df_gks.replace(to_replace="Idad And To", value="Trinidad and Tobago")
df_gks = df_gks.replace(to_replace="Turkmenista", value="Turkmenistan")
df_gks = df_gks.replace(to_replace="Uk", value="United Kingdom")
df_gks = df_gks.replace(to_replace="Usa", value="United States of America")
df_gks = df_gks.replace(to_replace="Minican Repu", value="Dominican Republic")

In [9]:
df_gks.head(10)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN
5,CS01220223,Embassy,Argentina,Fernandez Carolina Belen,Master,Konyang University,Sogang University,International relations,NaN
6,CS01220224,Embassy,Armenia,Hayrapetyan Laura,Master,Pai Chai University,Hankuk University Of Foreign Studies,Korean language and literature,Conditionalsuccessful candidate
7,CS01220225,Embassy,Armenia,Martirosyan Milena,Master,Busan University Of Foreignstudies,Ewha Womans University,International studies,NaN
8,CS01220226,Embassy,Australia,Gorey Michael Francis,Master,Exempt,Dongguk University,Film and digital media,NaN
9,CS01220227,Embassy,Austria,Wortha Samuel Roberto,Master,Exempt,Korea University,Media & communication,Scholarship deffered(6 months)


In [10]:
rango = range(0, df_gks.shape[0])  ### 
country_code=list(rango)
continent_code =list(rango)

for i in range(df_gks.shape[0]):
    country_code[i] = pc.country_name_to_country_alpha2(df_gks.loc[i,'Country'], cn_name_format="default")
    #print(country_code[i])
    
country_code = list(map(lambda item: item.replace("TL","TH"), country_code))
    
for i in range(df_gks.shape[0]):    
    continent_code[i] = pc.country_alpha2_to_continent_code(country_code[i])
    ##print(continent_code[i])    

In [11]:
df_gks['alpha2_country'] = country_code
df_gks['alpha2_cotinent'] = continent_code

In [34]:
geolocator = Nominatim(user_agent="pizza")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_gks['location'] = df_gks['Country'].apply(geocode)
df_gks['point'] = df_gks['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [64]:
##rango = range(0, df_gks.shape[0])  ### 
latitude_value=list(rango)
longitude_value =list(rango)

for i in range(df_gks.shape[0]):
    latitude_value[i]  = df_gks.loc[i,'point'][0] 
    longitude_value[i] = df_gks.loc[i,'point'][1]

In [65]:
df_gks['latitude'] = latitude_value
df_gks['longitude'] = longitude_value

In [93]:
df_gks['country_sum'] = df_gks.groupby('alpha2_country')['one'].transform('sum')
df_gks['proportion'] = df_gks['country_sum'] / df_gks.shape[0]

In [85]:
one =list(rango)

for i in range(df_gks.shape[0]):
    one[i] = 1
            
df_gks['one'] = one

##df_gks.drop('one', inplace=True, axis=1)

In [90]:
df_gks.head(5)

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent,location,point,latitude,longitude,one,country_sum
0,CS01220218,Embassy,Albania,Agalliu Enrik,Master,Seokyeong University,Kyungpook National University,Electronic and electrical engineering,NaN,AL,EU,"(Shqipëria, (41.000028, 19.9999619))","(41.000028, 19.9999619, 0.0)",41.000028,19.999962,1,1
1,CS01220219,Embassy,Algeria,Hadj Mohammed Hanane,Doctoral,Pai Chai University,Seoul National University,Earth and environmental sciences,NaN,DZ,AF,"(Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر, (28.0000272, 2.9...","(28.0000272, 2.9999825, 0.0)",28.000027,2.999983,1,4
2,CS01220220,Embassy,Algeria,Bourahla Kamelia,Master,Sunmoon University,Korea University,Business administration,NaN,DZ,AF,"(Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر, (28.0000272, 2.9...","(28.0000272, 2.9999825, 0.0)",28.000027,2.999983,1,4
3,CS01220221,Embassy,Angola,Pereira Job Judith Patricia,Master,Sunmoon University,Pusan National University,Dept. of food and resource economics,NaN,AO,AF,"(Angola, (-11.8775768, 17.5691241))","(-11.8775768, 17.5691241, 0.0)",-11.877577,17.569124,1,1
4,CS01220222,Embassy,Argentina,Centurion Natali Egle,Master,Keimyung University,Hankuk University Of Foreign Studies,International studies,NaN,AR,SA,"(Argentina, (-34.9964963, -64.9672817))","(-34.9964963, -64.9672817, 0.0)",-34.996496,-64.967282,1,2


In [143]:
df_gks.loc[df_gks['Application Track'] == 'R&D'] ##Seoul National University Of Scienceand Technology

,CS Number,Application Track,Country,Full Name (Family Given Middle),Program,Korean Language Institution,University,Department,Scholarship Status,alpha2_country,alpha2_cotinent,location,point,latitude,longitude,one,country_sum,proportion
1011,CS01221229,R&D,Indonesia,Lestari Windy Ayu,Doctoral,Gyeongsang Nationaluniversity,Gyeongsang National University,Materials engineering and convergencetechnology,NaN,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)",-2.483383,117.890285,1,99,0.082984
1012,CS01221230,R&D,Bangladesh,Sarkar Md Rubel,Master,Gyeongsang Nationaluniversity,Gyeongsang National University,Electronic engineering,NaN,BD,AS,"(বাংলাদেশ, (24.4769288, 90.2934413))","(24.4769288, 90.2934413, 0.0)",24.476929,90.293441,1,20,0.016764
1013,CS01221231,R&D,Indonesia,Zabrian Dirfan,Master,Gyeongsang Nationaluniversity,Gyeongsang National University,Materials engineering and convergencetechnology,NaN,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)",-2.483383,117.890285,1,99,0.082984
1014,CS01221232,R&D,Mongolia,Badamaa Batzorig,Doctoral,Gyeongsang Nationaluniversity,Gyeongsang National University,Engergy and mechanical engineering,NaN,MN,AS,"(Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ, (46.8250388, 103.8499...","(46.8250388, 103.8499736, 0.0)",46.825039,103.849974,1,59,0.049455
1015,CS01221233,R&D,Mongolia,Shirbazar Altankhundagakhutchin,Doctoral,Gyeongsang Nationaluniversity,Gyeongsang National University,Engergy and mechanical engineering,NaN,MN,AS,"(Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ, (46.8250388, 103.8499...","(46.8250388, 103.8499736, 0.0)",46.825039,103.849974,1,59,0.049455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,CS01221307,R&D,Indonesia,Syahputra Endry Wahyu,Master,Keimyung University,Keimyung University,Pharmacy,NaN,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)",-2.483383,117.890285,1,99,0.082984
1090,CS01221308,R&D,Indonesia,Ivani Ria Amelia,Master,Keimyung University,Keimyung University,Pharmacy,NaN,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)",-2.483383,117.890285,1,99,0.082984
1091,CS01221309,R&D,Indonesia,Wulandari Elina,Master,Keimyung University,Keimyung University,Pharmacy,NaN,ID,AS,"(Indonesia, (-2.4833826, 117.8902853))","(-2.4833826, 117.8902853, 0.0)",-2.483383,117.890285,1,99,0.082984
1092,CS01221310,R&D,Pakistan,Anwar Rubab,Master,Korea University Oftechnology And Education,Korea University Of Technology Andeducation (K...,Department of computer science and engineering,NaN,PK,AS,"(پاکستان, (30.3308401, 71.247499))","(30.3308401, 71.247499, 0.0)",30.330840,71.247499,1,19,0.015926


In [126]:
df_country = df_gks[['Country','alpha2_country','latitude','longitude','country_sum','proportion']].copy()

In [127]:
df_country = df_country.drop_duplicates()

In [128]:
len(df_country)

129

In [131]:
df_country.head(10)

,Country,alpha2_country,latitude,longitude,country_sum,proportion
0,Albania,AL,41.000028,19.999962,1,0.000838
1,Algeria,DZ,28.000027,2.999983,4,0.003353
3,Angola,AO,-11.877577,17.569124,1,0.000838
4,Argentina,AR,-34.996496,-64.967282,2,0.001676
6,Armenia,AM,40.769627,44.673665,3,0.002515
8,Australia,AU,-24.776109,134.755000,1,0.000838
9,Austria,AT,47.593970,14.124560,1,0.000838
10,Azerbaijan,AZ,40.393629,47.787251,19,0.015926
16,Bahrain,BH,26.155125,50.534461,3,0.002515
19,Bangladesh,BD,24.476929,90.293441,20,0.016764


In [ ]:
df_gks.drop('one', inplace=True, axis=1)

In [135]:
#installation
#!pip install folium
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_country)):
        lat =  df_country.iloc[i]['latitude']
        long = df_country.iloc[i]['longitude']
        radius=5
        popup_text = """Country : {}<br>
                    %of Users : {}<br>"""
        popup_text = popup_text.format(df_country.iloc[i]['alpha2_country'],
                                   df_country.iloc[i]['country_sum']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [39]:
from pathlib import Path  
filepath = Path('D:/Coreano/jeong/koreanLanguage/salida.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_gks.to_csv(filepath)  